In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 592.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("what is a confirmist")
print(result.content)

**Definition:**

A confirmist is an individual who tends to conform to the beliefs and opinions of others, particularly those in a position of authority or within a social group.

**Characteristics:**

* **High Need for Approval:** Confirmists have a strong desire to be accepted and approved of by others.
* **Low Self-Esteem:** They may lack confidence in their own judgment and abilities.
* **Social Anxiety:** Confirmists may be anxious about being judged or rejected by others.
* **Groupthink:** They are highly susceptible to group pressure and may sacrifice their own beliefs to conform.
* **Avoidance of Conflict:** Confirmists prefer to avoid confrontation and go along with the crowd to maintain harmony.
* **Cognitive Dissonance:** They may experience discomfort when their own beliefs conflict with the views of others.

**Motivations:**

* **Social Acceptance:** Confirmists believe that conforming will help them fit in and avoid social disapproval.
* **Fear of Isolation:** They may fe

In [ ]:
from crewai import Agent, Task, Crew


In [ ]:
# Multi-agnet to write article
# Three total agents with one specific task at hand
#     - Plan
#     - Write
#     - Edit

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True,
    llm=llm
)

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
    llm=llm
)

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
    llm=llm
)

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor,
    llm=llm
)

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
**Final Answer:**

**Content Plan: Artificial Intelligence**

**1. Audience Analysis**

* **Target Audience:**
    * Business professionals, tech enthusiasts, and general readers interested in AI's impact on various industries.
* **Interests:**
    * AI advancements, industry applications, ethical considerations.
* **Pain Points:**
    * Confusion about AI's technical complexities.
    * Uncertainty about AI's impact on jobs and society.

**2. Content Outline**

**Introduction:**
* Definition and overview of Artificial Intelligence (AI).
* Brief history 

In [ ]:
from IPython.display import Markdown
Markdown(result)

**

**Title: Artificial Intelligence: Revolutionizing Industries and Shaping the Future**

**Introduction**

Artificial Intelligence (AI) has emerged as a transformative force, revolutionizing industries and reshaping the way we live. From its humble beginnings to its current state-of-the-art capabilities, AI has captured the attention of business leaders, tech enthusiasts, and society at large. In this blog post, we will delve into the latest trends, key players, and ethical considerations surrounding AI, exploring its profound impact on various industries and its implications for the future.

**Key Points**

**Section 1: The Rise of AI and Its Applications**

AI has witnessed remarkable advancements in recent years, particularly in the fields of machine learning, deep learning, and natural language processing. These advancements have paved the way for AI-driven automation, robotics, and innovative applications across industries. In healthcare, AI assists in diagnostics and personalized treatments, while in finance, it enhances fraud detection and risk management. Moreover, AI-powered self-driving cars and traffic optimization are revolutionizing the transportation sector.

**Section 2: The AI Landscape**

Leading tech giants such as Google, Microsoft, and Amazon are at the forefront of AI development and deployment. Startups and research institutions are also driving innovation, pushing the boundaries of AI capabilities. Governments and regulatory bodies play a crucial role in shaping AI policies and guidelines, ensuring responsible development and deployment.

**Section 3: AI in the News**

The AI industry is constantly evolving, with new breakthroughs and developments emerging regularly. Recent investments and partnerships have accelerated AI research and applications. Ethical debates and societal implications of AI have also gained prominence, raising questions about job displacement, societal biases, and the potential misuse of AI technologies.

**Section 4: AI's Industry-Specific Impact**

AI is transforming industries in profound ways. In healthcare, it improves patient outcomes through AI-assisted diagnostics and personalized treatments. In finance, AI enhances fraud detection and risk management, increasing efficiency and security. The transportation sector benefits from AI-powered self-driving cars and traffic optimization, reducing accidents and improving mobility.

**Section 5: Ethical Considerations and the Future of AI**

While AI offers immense potential, ethical considerations and future implications must be carefully examined. Concerns about AI job displacement and societal biases need to be addressed. Responsible development and deployment of AI technologies are paramount. Moreover, exploring the potential of AI to solve global challenges and improve human lives is essential.

**Call to Action**

As AI continues to evolve, it is imperative to stay informed about its advancements and implications. Further research and discussion on the ethical and societal impact of AI are crucial. We encourage readers to delve deeper into the world of AI, explore its applications, and engage in thoughtful discussions about its future.

**Conclusion**

Artificial Intelligence has become an integral part of our present and holds transformative power for the future. Its advancements have brought about remarkable changes across industries, offering both opportunities and challenges. As we navigate the rapidly evolving AI landscape, it is essential to embrace responsible development, address ethical concerns, and harness the potential of AI for the betterment of society.